In [ ]:
pip install optuna

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 308 kB 23.3 MB/s 
     |████████████████████████████████| 209 kB 53.2 MB/s 
     |████████████████████████████████| 81 kB 8.6 MB/s 
     |████████████████████████████████| 78 kB 6.6 MB/s 
     |████████████████████████████████| 146 kB 53.5 MB/s 
     |████████████████████████████████| 112 kB 48.0 MB/s 
     |████████████████████████████████| 49 kB 6.0 MB/s 
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11137 sha256=117a2e2720122fbf6d3c1752183af5137fbf786fb4870ca14720b426978028ae
  Stored in directory: /root/.cache/pip/wheels/9f/18/84/8f69f8b08169c7bae2dde6bd7daf0c19fca8c8e500ee620a28
Successfully built pyperclip


In [ ]:
import optuna
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [ ]:
import torch

class Dataset(torch.utils.data.Dataset):

    def __init__(self, df):
        self.labels = [0 if label == 0 else 1 for label in df['HeartDisease']        self.features = df.drop(columns=['HeartDisease'], axis=1).values.tolist()

    def classes(self):
        return self.labels

    def __len__(self):
        return len(self.labels)

    def get_batch_labels(self, idx):
        return np.array(self.labels[idx])

    def get_batch_features(self, idx):
        return np.array(self.features[idx])

    def __getitem__(self, idx):
        batch_features = self.get_batch_features(idx)
        batch_y = self.get_batch_labels(idx)

        return batch_features, batch_y

In [ ]:
# Build neural network model
def build_model(params):
    
    in_features = 20
    
    return nn.Sequential(
    
        nn.Linear(in_features, params['n_unit']),
        nn.LeakyReLU(),
        nn.Linear(params['n_unit'], 2),
        nn.LeakyReLU()
        
    )
 
 
# Train and evaluate the accuarcy of neural network model
def train_and_evaluate(param, model):
    
    df = pd.read_csv('heart.csv')
    df = pd.get_dummies(df)
    
    train_data, val_data = train_test_split(df, test_size = 0.2, random_state = 42)
    train, val = Dataset(train_data), Dataset(val_data)

    train_dataloader = torch.utils.data.DataLoader(train, batch_size=2, shuffle=True)
    val_dataloader = torch.utils.data.DataLoader(val, batch_size=2)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    criterion = nn.CrossEntropyLoss()
    optimizer = getattr(optim, param['optimizer'])(model.parameters(), lr= param['learning_rate'])

    if use_cuda:
            model = model.cuda()
            criterion = criterion.cuda()

    for epoch_num in range(EPOCHS):

            total_acc_train = 0
            total_loss_train = 0

            for train_input, train_label in train_dataloader:

                train_label = train_label.to(device)
                train_input = train_input.to(device)

                output = model(train_input.float())
                
                batch_loss = criterion(output, train_label.long())
                total_loss_train += batch_loss.item()
                
                acc = (output.argmax(dim=1) == train_label).sum().item()
                total_acc_train += acc

                model.zero_grad()
                batch_loss.backward()
                optimizer.step()
            
            total_acc_val = 0
            total_loss_val = 0

            with torch.no_grad():

                for val_input, val_label in val_dataloader:

                    val_label = val_label.to(device)
                    val_input = val_input.to(device)
                    output = model(val_input.float())

                    batch_loss = criterion(output, val_label.long())
                    total_loss_val += batch_loss.item()
                    
                    acc = (output.argmax(dim=1) == val_label).sum().item()
                    total_acc_val += acc
            
            accuracy = total_acc_val/len(val_data)

    return accuracy
  
 # Define a set of hyperparameter values, build the model, train the model, and evaluate the accuracy 
def objective(trial):

     params = {
              'learning_rate': trial.suggest_loguniform('learning_rate', 1e-5, 1e-1),
              'optimizer': trial.suggest_categorical("optimizer", ["Adam", "RMSprop", "SGD"]),
              'n_unit': trial.suggest_int("n_unit", 4, 18)
              }
    
     model = build_model(params)
    
     accuracy = train_and_evaluate(params, model)

     return accuracy


In [ ]:
EPOCHS = 30
    
study = optuna.create_study(direction="maximize", sampler=optuna.samplers.TPESampler())
study.optimize(objective, n_trials=30)

[I 2022-07-10 10:52:57,762] A new study created in memory with name: no-name-a596dd82-4c8b-422a-bb05-0174d1612cf0
[I 2022-07-10 10:53:08,984] Trial 0 finished with value: 0.875 and parameters: {'learning_rate': 0.03676061875784552, 'optimizer': 'Adam', 'n_unit': 15}. Best is trial 0 with value: 0.875.
[I 2022-07-10 10:53:18,980] Trial 1 finished with value: 0.8695652173913043 and parameters: {'learning_rate': 0.0014369830369843329, 'optimizer': 'RMSprop', 'n_unit': 11}. Best is trial 0 with value: 0.875.
[I 2022-07-10 10:53:27,523] Trial 2 finished with value: 0.5597826086956522 and parameters: {'learning_rate': 0.0008001527364908191, 'optimizer': 'SGD', 'n_unit': 16}. Best is trial 0 with value: 0.875.
[I 2022-07-10 10:53:36,958] Trial 3 finished with value: 0.7065217391304348 and parameters: {'learning_rate': 0.06870267218386751, 'optimizer': 'SGD', 'n_unit': 14}. Best is trial 0 with value: 0.875.
[I 2022-07-10 10:53:47,075] Trial 4 finished with value: 0.875 and parameters: {'learn

In [ ]:
best_trial = study.best_trial

for key, value in best_trial.params.items():
    print("{}: {}".format(key, value))

learning_rate: 0.031014465283528837
optimizer: Adam
n_unit: 15


In [ ]:
# Build a model by implementing define-by-run design from Optuna
def build_model_custom(trial):
    
    n_layers = trial.suggest_int("n_layers", 1, 3)
    layers = []

    in_features = 20
    
    for i in range(n_layers):
        
        out_features = trial.suggest_int("n_units_l{}".format(i), 4, 18)
        
        layers.append(nn.Linear(in_features, out_features))
        layers.append(nn.LeakyReLU())

        in_features = out_features
        
    layers.append(nn.Linear(in_features, 2))
    layers.append(nn.LeakyReLU())
    
    return nn.Sequential(*layers)

# Train and evaluate the accuracy of neural network with the addition of pruning mechanism
def train_and_evaluate(param, model, trial):
    
    df = pd.read_csv('heart.csv') 
    df = pd.get_dummies(df)
    
    train_data, val_data = train_test_split(df, test_size = 0.2, random_state = 42)
    train, val = Dataset(train_data), Dataset(val_data)

    train_dataloader = torch.utils.data.DataLoader(train, batch_size=2, shuffle=True)
    val_dataloader = torch.utils.data.DataLoader(val, batch_size=2)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")
    criterion = nn.CrossEntropyLoss()
    optimizer = getattr(optim, param['optimizer'])(model.parameters(), lr= param['learning_rate'])

    if use_cuda:

            model = model.cuda()
            criterion = criterion.cuda()

    for epoch_num in range(EPOCHS):

            total_acc_train = 0
            total_loss_train = 0

            for train_input, train_label in train_dataloader:

                train_label = train_label.to(device)
                train_input = train_input.to(device)

                output = model(train_input.float())
                
                batch_loss = criterion(output, train_label.long())
                total_loss_train += batch_loss.item()
                
                acc = (output.argmax(dim=1) == train_label).sum().item()
                total_acc_train += acc

                model.zero_grad()
                batch_loss.backward()
                optimizer.step()
            
            total_acc_val = 0
            total_loss_val = 0

            with torch.no_grad():

                for val_input, val_label in val_dataloader:
                  val_label = val_label.to(device)
                  val_input = val_input.to(device)

                  output = model(val_input.float())

                  batch_loss = criterion(output, val_label.long())
                  total_loss_val += batch_loss.item()
                    
                  acc = (output.argmax(dim=1) == val_label).sum().item()
                  total_acc_val += acc
            
            accuracy = total_acc_val/len(val_data)
            
            # Add prune mechanism
            trial.report(accuracy, epoch_num)

            if trial.should_prune():
                raise optuna.exceptions.TrialPruned()

    return accuracy
  
# Define a set of hyperparameter values, build the model, train the model, and evaluate the accuracy
def objective(trial):

     params = {
              'learning_rate': trial.suggest_loguniform('learning_rate', 1e-5, 1e-1),
              'optimizer': trial.suggest_categorical("optimizer", ["Adam", "RMSprop", "SGD"]),
              }
    
     model = build_model_custom(trial)

     accuracy = train_and_evaluate(params, model, trial)

     return accuracy
EPOCHS = 30
    
study = optuna.create_study(direction="maximize", sampler=optuna.samplers.TPESampler(), pruner=optuna.pruners.MedianPruner())
study.optimize(objective, n_trials=30)

[I 2022-07-10 11:44:32,029] A new study created in memory with name: no-name-9d1917b4-c2fc-4ed8-9398-8cf9bf124c2f
[I 2022-07-10 11:44:47,495] Trial 0 finished with value: 0.8478260869565217 and parameters: {'learning_rate': 0.012509585473555804, 'optimizer': 'Adam', 'n_layers': 3, 'n_units_l0': 13, 'n_units_l1': 9, 'n_units_l2': 5}. Best is trial 0 with value: 0.8478260869565217.
[I 2022-07-10 11:44:58,587] Trial 1 finished with value: 0.5760869565217391 and parameters: {'learning_rate': 0.0023085552219187155, 'optimizer': 'SGD', 'n_layers': 3, 'n_units_l0': 6, 'n_units_l1': 6, 'n_units_l2': 7}. Best is trial 0 with value: 0.8478260869565217.
[I 2022-07-10 11:45:11,891] Trial 2 finished with value: 0.6956521739130435 and parameters: {'learning_rate': 3.586922875594401e-05, 'optimizer': 'Adam', 'n_layers': 2, 'n_units_l0': 7, 'n_units_l1': 17}. Best is trial 0 with value: 0.8478260869565217.
[I 2022-07-10 11:45:23,373] Trial 3 finished with value: 0.6793478260869565 and parameters: {'le

In [ ]:
best_trial = study.best_trial

for key, value in best_trial.params.items():
    print("{}: {}".format(key, value))

learning_rate: 0.0006723131071157847
optimizer: Adam
n_layers: 2
n_units_l0: 10
n_units_l1: 11


In [ ]:
optuna.visualization.plot_intermediate_values(study)

In [ ]:
optuna.visualization.plot_optimization_history(study)

In [ ]:
optuna.visualization.plot_parallel_coordinate(study)


In [ ]:
optuna.visualization.plot_param_importances(study)